# 数据清洗

到目前为止，我们还没有处理过那些样式不规范的数据，要么是使用样式规范的数据源，
要么就是彻底放弃样式不符合我们预期的数据。但是在网络数据采集中，你通常无法对采
集的数据样式太挑剔。

由于错误的标点符号、大小写字母不一致、断行和拼写错误等问题，零乱的数据（dirty
data）是网络中的大问题。本章将介绍一些工具和技术，通过改变代码的编写方式，帮你
从源头控制数据零乱的问题，并且对已经进入数据库的数据进行清洗。

### 1　编写代码清洗数据

和写代码处理异常一样，你也应该学习编写预防型代码来处理意外情况。

在语言学里有一个模型叫 n-gram，表示文字或语言中的 n 个连续的单词组成的序列。在进行自然语言分析时，使用 n-gram 或者寻找常用词组，可以很容易地把一句话分解成若干个文字片段。

这一节我们将重点介绍如何获取格式合理的 n-gram，并不用它们做任何分析。在第8章，我们再用2-gram和3-gram来做文本摘要提取和语法分析。

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

def ngrams(input, n):
    input = input.split(" ")
    output = []
    for i in range(len(input) - n +1):
        output.append(input[i: i+n])
    return output

html = urlopen("http://en.wikipedia.org/wiki/Python_(programming_language)")
bsObj = BeautifulSoup(html, 'html.parser')
content = bsObj.find("div",{"id":"mw-content-text"}).get_text()
ngrams = ngrams(content, 2)
print(ngrams)
print("2-grams out is:" + str(len(ngrams)))

[['\nPython\n\n\n\n\nParadigm\nObject-oriented,', 'imperative,'], ['imperative,', 'functional,'], ['functional,', 'procedural,'], ['procedural,', 'reflective\n\n\nDesigned\xa0by\nGuido'], ['reflective\n\n\nDesigned\xa0by\nGuido', 'van'], ['van', 'Rossum\n\n\nDeveloper\nPython'], ['Rossum\n\n\nDeveloper\nPython', 'Software'], ['Software', 'Foundation\n\n\nFirst\xa0appeared\n20\xa0February'], ['Foundation\n\n\nFirst\xa0appeared\n20\xa0February', '1991;'], ['1991;', '27'], ['27', 'years'], ['years', 'ago\xa0(1991-02-20)[1]\n\n\n\n\n\nStable'], ['ago\xa0(1991-02-20)[1]\n\n\n\n\n\nStable', 'release\n\n3.6.4'], ['release\n\n3.6.4', '/'], ['/', '19\xa0December'], ['19\xa0December', '2017;'], ['2017;', '3'], ['3', 'months'], ['months', 'ago\xa0(2017-12-19)[2]\n2.7.14'], ['ago\xa0(2017-12-19)[2]\n2.7.14', '/'], ['/', '16\xa0September'], ['16\xa0September', '2017;'], ['2017;', '6'], ['6', 'months'], ['months', 'ago\xa0(2017-09-16)[3]\n\n\n\nPreview'], ['ago\xa0(2017-09-16)[3]\n\n\n\nPreview', 'r

ngrams 函数把一个待处理的字符串分成单词序列（假设所有单词按照空格分开），然后增加到 n-gram 模型（本例中是 2-gram）里形成以每个单词开始的二元数组。

因为每个单词（除了最后一个单词）都要创建一个 2-gram 序列，所以这个词条里共有 7411 个 2-gram 序列。这并不是一个非常便于管理的数据集！


让我们首先用一些正则表达式来移除转义字符（\n），再把 Unicode 字符过滤掉。我们可以通过下面的函数对之前输出的结果进行清理：
```python
import re
def ngrams(input, n):
    content = re.sub('\n+', " ", content)
    content = re.sub(' +', " ", content)
    content = bytes(content, "UTF-8")
    content = content.decode("ascii", "ignore")
    print(content)
    input = input.split(' ')
    output = []
    for i in range(len(input)-n+1):
        output.append(input[i:i+n])
    return output
```

这里首先把内容中的换行符（或者多个换行符）替换成空格，然后把连续的多个空格替换成一个空格，确保所有单词之间只有一个空格。最后，把内容转换成 UTF-8 格式以消除转义字符。

这几步已经可以大大改善输出结果了，但是还有一些问题：
```python
['Pythoneers.[43][44]', 'Syntax'], ['7', '/'], ['/', '3'], ['3', '=='], ['==', '2']
```

因此，需要再增加一些规则来处理数据。我们还可以制定一些规则让数据变得更规范：
- 剔除单字符的“单词”，除非这个字符是“i”或“a”；
- 剔除维基百科的引用标记（方括号包裹的数字，如 [1]）；
- 剔除标点符号（注意：这个规则有点儿矫枉过正，在第 9 章我们将详细介绍，本例暂时这样处理）。

#### 现在“清洗任务”列表变得越来越长，让我们把规则都移出来，单独建一个函数，就叫cleanInput：

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import string

def cleanInput(input):
    input = re.sub('\n+', " ", input)
    input = re.sub('\[[0-9]*\]', "", input)
    input = re.sub(' +', " ", input)
    input = bytes(input, "UTF-8")
    input = input.decode("ascii", "ignore")
    cleanInput = []
    input = input.split(' ')
    for item in input:
        item = item.strip(string.punctuation)
        if len(item) > 1 or (item.lower() == 'a' or item.lower() == 'i'):
            cleanInput.append(item)
    return cleanInput

def ngrams(input, n):
    input = cleanInput(input)
    output = []
    for i in range(len(input)-n+1):
        output.append(input[i:i+n])
    return output

In [5]:
html = urlopen("http://en.wikipedia.org/wiki/Python_(programming_language)")
bsObj = BeautifulSoup(html, 'html.parser')
content = bsObj.find("div",{"id":"mw-content-text"}).get_text()
ngrams = ngrams(content, 2)
print(ngrams)
print("2-grams out is:" + str(len(ngrams)))

[['Python', 'Paradigm'], ['Paradigm', 'Object-oriented'], ['Object-oriented', 'imperative'], ['imperative', 'functional'], ['functional', 'procedural'], ['procedural', 'reflective'], ['reflective', 'Designedby'], ['Designedby', 'Guido'], ['Guido', 'van'], ['van', 'Rossum'], ['Rossum', 'Developer'], ['Developer', 'Python'], ['Python', 'Software'], ['Software', 'Foundation'], ['Foundation', 'Firstappeared'], ['Firstappeared', '20February'], ['20February', '1991'], ['1991', '27'], ['27', 'years'], ['years', 'ago(1991-02-20'], ['ago(1991-02-20', 'Stable'], ['Stable', 'release'], ['release', '3.6.4'], ['3.6.4', '19December'], ['19December', '2017'], ['2017', 'months'], ['months', 'ago(2017-12-19'], ['ago(2017-12-19', '2.7.14'], ['2.7.14', '16September'], ['16September', '2017'], ['2017', 'months'], ['months', 'ago(2017-09-16'], ['ago(2017-09-16', 'Preview'], ['Preview', 'release'], ['release', '3.6.5rc1'], ['3.6.5rc1', 'and'], ['and', '3.7.0b1'], ['3.7.0b1', '2018'], ['2018', 'Typing'], ['T

### 数据标准化

还用之前的 n-gram 示例，让我们在上面增加一些数据标准化特征.


这段代码有一个明显的问题，就是输出结果中包含太多重复的 2-gram 序列。程序把每个2-gram 序列都加入了列表，没有统计过序列的频率。掌握 2-gram 序列的频率，而不只是知道某个序列是否存在，这不仅很有意思，而且有助于对比不同的数据清洗和数据标准化算法的效果。如果数据标准化成功了，那么唯一的 n-gram 序列数量就会减少，而 n-gram序列的总数（任何一个 n-gram 序列和与之重复的序列被看成一个 n-gram 序列）不变。也就是说，同样数量的 n-gram 序列，经过去重之后“容量”（bucket）会减少。

不过 Python 的字典是无序的，不能像数组一样直接对 n-gram 序列频率进行排序。字典内部元素的位置不是固定的，排序之后再次使用时还是会变化，除非你把排序过的字典里的值复制到其他类型中进行排序。在 Python 的 collections 库里面有一个 OrderedDict 可以解决这个问题：

```python

from collections import OrderedDict
...
ngrams = ngrams(content, 2)
ngrams = OrderedDict(sorted(ngrams.items(), key=lambda t: t[1], reverse=True))
print(ngrams)
```

除了这些，还需要再考虑一下，自己计划为数据标准化的进一步深入再投入多少计算能力。很多单词在不同的环境里会使用不同的拼写形式，其实都是等价的，但是为了解决这种等价关系，你需要对每个单词进行检查，判断是否和其他单词有等价关系。

### 2　数据存储后再清洗

对于编写代码清洗数据，你能做或想做的事情只有这些。除此之外，你可能还需要处理一些别人创建的数据库，或者要对一个之前没接触过的数据库进行清洗。

很多程序员遇到这种情况的自然反应就是“写个脚本”，当然这也是一个很好的解决方法。但是，还有一些第三方工具，像 OpenRefine，不仅可以快速简单地清理数据，还可以让非编程人员轻松地看见和使用你的数据。